In [8]:
import os

**Setup and clone darknet**

In [26]:
if not os.path.exists('darknet'):
    !git clone https://github.com/AlexeyAB/darknet
    %cd /content/darknet
    !sed -i 's/OPENCV=0/OPENCV=1/' Makefile
    !sed -i 's/GPU=0/GPU=1/' Makefile
    !sed -i 's/CUDNN=0/CUDNN=1/' Makefile
    !sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
    !make
    %cd /content
    !pip install -q torch_snippets
    !wget --quiet https://www.dropbox.com/s/agmzwk95v96ihic/open-images-bus-trucks.tar.xz
    !tar -xf open-images-bus-trucks.tar.xz
    !rm open-images-bus-trucks.tar.xz
    !wget --quiet https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.weights

**Test darknet installation**

In [44]:
!./darknet detector test darknet/cfg/coco.data darknet/cfg/yolov4.cfg yolov4.weights darknet/data/person.jpg

 GPU isn't used 
 OpenCV isn't used - data augmentation will be slow 
Couldn't open file: darknet/cfg/coco.data


**Setting up the dataset format**

In [32]:
# Cell 3: Set up Dataset Format
%%writefile data/obj.names
bus
truck

%%writefile data/obj.data
classes = 2
train = data/train.txt
valid = data/val.txt
names = data/obj.names
backup = backup/

!mkdir -p data/obj
!cp -r open-images-bus-trucks/images/* data/obj/
!cp open-images-bus-trucks/yolo_labels/all/train.txt data/
!cp open-images-bus-trucks/yolo_labels/all/val.txt data/
!cp open-images-bus-trucks/yolo_labels/all/labels/*.txt data/obj/

Overwriting data/obj.names


**Configuring the architecture**

In [33]:
# Cell 4: Configure the Architecture
!cp darknet/cfg/yolov4-tiny-custom.cfg cfg/yolov4-tiny-bus-trucks.cfg
!sed -i 's/max_batches = 500200/max_batches=4000/' cfg/yolov4-tiny-bus-trucks.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov4-tiny-bus-trucks.cfg
!sed -i 's/steps=400000,450000/steps=3200,3600/' cfg/yolov4-tiny-bus-trucks.cfg
!sed -i 's/classes=80/classes=2/g' cfg/yolov4-tiny-bus-trucks.cfg
!sed -i 's/filters=255/filters=21/g' cfg/yolov4-tiny-bus-trucks.cfg
!sed -i 's/filters=57/filters=33/g' cfg/yolov4-tiny-bus-trucks.cfg

cp: cannot stat 'darknet/cfg/yolov4-tiny-custom.cfg': Not a directory


**Download pre-trained weights**

In [35]:
# Cell 5: Download Pre-trained Weights
!wget --quiet https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.conv.29
!cp yolov4-tiny.conv.29 /content/darknet/

Train the model

In [39]:
!./darknet detector train data/obj.data cfg/yolov4-tiny-bus-trucks.cfg yolov4-tiny.conv.29 -dont_show -mapLastAt

 GPU isn't used 
 OpenCV isn't used - data augmentation will be slow 
yolov4-tiny-bus-trucks
mini_batch = 1, batch = 166, time_steps = 1, train = 1 
   layer   filters  size/strd(dil)      input                output
   0 conv     32       3 x 3/ 2    416 x 416 x   3 ->  208 x 208 x  32 0.075 BF
   1 conv     64       3 x 3/ 2    208 x 208 x  32 ->  104 x 104 x  64 0.399 BF
   2 conv     64       3 x 3/ 1    104 x 104 x  64 ->  104 x 104 x  64 0.797 BF
   3 route  2 		                       1/2 ->  104 x 104 x  32 
   4 conv     32       3 x 3/ 1    104 x 104 x  32 ->  104 x 104 x  32 0.199 BF
   5 conv     32       3 x 3/ 1    104 x 104 x  32 ->  104 x 104 x  32 0.199 BF
   6 route  5 4 	                           ->  104 x 104 x  64 
   7 conv     64       1 x 1/ 1    104 x 104 x  64 ->  104 x 104 x  64 0.089 BF
   8 route  2 7 	                           ->  104 x 104 x 128 
   9 max                2x 2/ 2    104 x 104 x 128 ->   52 x  52 x 128 0.001 BF
  10 conv    128       3 x 3/

Test the model on custom images

In [37]:
# Cell 7: Test the Model on Custom Images
from torch_snippets import Glob, stem, show, read
import os

# Create a directory for custom images
!mkdir -p custom_images

# TODO: Upload your images to the 'custom_images' directory

image_paths = Glob('custom_images/*')
for f in image_paths:
    !./darknet/darknet detector test data/obj.data cfg/yolov4-tiny-bus-trucks.cfg \
    backup/yolov4-tiny-bus-trucks_4000.weights {f} -dont_show
    !mv predictions.jpg {os.path.splitext(f)[0]}_pred.jpg

for i in Glob('custom_images/*_pred.jpg', silent=True):
    show(read(i, 1), sz=20)